# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import findspark

In [2]:
findspark.init('/home/coffey/spark-2.2.0-bin-hadoop2.7')

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('logistic').getOrCreate()

In [5]:
df = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [6]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [7]:
df.groupBy('Company').count().show()

+--------------------+-----+
|             Company|count|
+--------------------+-----+
|Miller, Johnson a...|    1|
|Hunter, Reyes and...|    1|
|          Obrien PLC|    1|
|            Soto PLC|    2|
|            Todd LLC|    1|
|Smith, Marshall a...|    1|
|           Smith PLC|    1|
|          Hall Group|    1|
|Freeman, Lam and ...|    1|
|       Smith-Carroll|    1|
|Hall, Hernandez a...|    1|
|          Cannon Inc|    1|
|        White-Dennis|    1|
|Wilson, Collins a...|    1|
|Jennings, Gates a...|    1|
|     Campbell-Willis|    1|
|    Martinez-Roberts|    1|
|        Robinson PLC|    1|
|          Barton Inc|    1|
|Hernandez, Middle...|    1|
+--------------------+-----+
only showing top 20 rows



#since it is looks like very customer comes from different company, 
#I will not consider 'Company' is one of the features

In [8]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [9]:
my_cols = df.select(['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date','Churn'])

In [15]:
# my_cols.describe().show()

In [12]:
#This time I only want to focus on the year when they onboard
from pyspark.sql.functions import year

In [13]:
df = my_cols.withColumn("Year_onboard", year(df['Onboard_date']))

In [14]:
df.show(4)

+----+--------------+---------------+-----+---------+-------------------+-----+------------+
| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|Churn|Year_onboard|
+----+--------------+---------------+-----+---------+-------------------+-----+------------+
|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|    1|        2013|
|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|    1|        2013|
|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|    1|        2016|
|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|    1|        2014|
+----+--------------+---------------+-----+---------+-------------------+-----+------------+
only showing top 4 rows



In [16]:
df.describe().show()

+-------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+------------------+
|summary|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|              Churn|      Year_onboard|
+-------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+------------------+
|  count|              900|              900|               900|              900|               900|                900|               900|
|   mean|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|0.16666666666666666|2010.8011111111111|
| stddev|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| 0.3728852122772358|3.2072288498508783|
|    min|             22.0|            100.0|                 0|              1.0|               3.0|                  0|              2006|
|    max|    

#now I'm going to indexer and onehotcoder 'Account_Manager'

In [17]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, VectorIndexer,OneHotEncoder, StringIndexer

In [18]:
AM_encoder = OneHotEncoder(inputCol='Account_Manager', outputCol='AM_vec')

YO_indexer = StringIndexer(inputCol='Year_onboard', outputCol='YO_index')
YO_encoder = OneHotEncoder(inputCol='YO_index', outputCol='YO_vec')

In [19]:
df.printSchema()

root
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- Year_onboard: integer (nullable = true)



In [20]:
assembler = VectorAssembler(
    inputCols=['Age', 'Total_Purchase', 'Years', 'Num_Sites', 'YO_vec','AM_vec'], 
    outputCol='features')

In [21]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [22]:
lr = LogisticRegression(featuresCol='features', labelCol='Churn')

In [23]:
pipeline = Pipeline(stages=[AM_encoder, YO_indexer, YO_encoder, assembler, lr])

In [24]:
train_data, test_data = df.randomSplit([0.7, 0.3])

In [25]:
fit_model = pipeline.fit(train_data)

In [26]:
results = fit_model.transform(test_data)

In [29]:
results.select(['rawPrediction', 'prediction', 'Churn']).show(5)

+--------------------+----------+-----+
|       rawPrediction|prediction|Churn|
+--------------------+----------+-----+
|[5.40141035907026...|       0.0|    0|
|[1.28811844137088...|       0.0|    1|
|[7.15262796814326...|       0.0|    0|
|[5.96595525605424...|       0.0|    0|
|[2.19698250960033...|       0.0|    0|
+--------------------+----------+-----+
only showing top 5 rows



In [30]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',labelCol='Churn')

In [31]:
auc = my_eval.evaluate(results)

In [32]:
auc

0.8966697502312678

# predictio on new data

In [35]:
new_customers = spark.read.csv('new_customers.csv', inferSchema=True, header=True)

In [36]:
new_customers = new_customers.withColumn("Year_onboard", year('Onboard_date'))

In [37]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Year_onboard: integer (nullable = true)



In [47]:
nc_data = new_customers.select(['Age',
 'Total_Purchase','Account_Manager','Years',
 'Num_Sites','Year_onboard','Company'])

In [39]:
pipeline2 = Pipeline(stages=[AM_encoder, YO_indexer, YO_encoder, assembler, lr])

In [41]:
df.printSchema()

root
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- Year_onboard: integer (nullable = true)



In [42]:
fit_model2 = pipeline2.fit(df)

In [48]:
my_res = fit_model2.transform(nc_data)

In [50]:
my_res.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



# Results: According to this results, I will notify the client 'Cannon_benson', 'Barron-Robertson', 'Sexton-Golden','Parks-Robbins' has high possibility churn, and we should do something to make them stay.